### **INTRODUÇÃO**

Este notebook traz raspagem, tratamento e análise de dados de internação por Covid-19 e Síndrome Respiratória Aguda Grave (SRAG) em leitos infantis  de Enfermaria e UTI no Ceará a partir da API do IntegraSUS, plataforma da Secretaria da Saúde do Estado (Sesa). O foco é avaliar os dados relativos aos leitos infantis no atual cenário epidemiológico no Ceará, uma vez que [atendimentos pediátricos têm aumentado nas Unidades de Pronto Atendimento (UPAs) de Fortaleza, principalmente por conta de síndromes gripais e respiratórias](https://www.opovo.com.br/noticias/fortaleza/2022/04/25/atendimentos-pediatricos-aumentam-155-upa-fortaleza-sindrome-respiratoria.html).

A partir dos dados de leitos ativos e leitos ocupados, calculou-se a média móvel de 7 dias para cada um deles desde o início da pandemia de Covid-19 no Estado. Com foco na média móvel, o objetivo é evitar trabalhar apenas com a taxa de ocupação (apesar de também calcularmos esse valor), uma vez que pode haver possíveis distorções pela variação causada pela abertura ou pelo fechamento de leitos ao longo dos dias.

A análise é feita em Python, e este documento traz os seguintes tópicos:

1. [Coletando dados](#Coleta)
2. [Lendo dados coletados](#Leitura)
3. [Criando novas variáveis para análises de leitos infantis](#Variaveis)
4. [Tratamento](#Tratamento)
5. [Análises](#Analise)\
    5.1 [Período após a identificação da Ômicron no Estado do Ceará](#Omicron)\
    5.2 [Identificando hospitais com as internações](#Hospitais)
6. [Salvar arquivos em .csv](#Arquivo)


Para reporduzir o passo a passo do que foi feito nesse notebook:
- utilize o arquivo `20220425_21h12m_dados_internacao_total.csv`, disponível na pasta "arquivos gerados"
- após importar as bibliotecas, execute o notebook a partir do item 2. [Lendo dados coletados](#Leitura)


**EXPEDIENTE**

**Edição DATADOC**

Thays Lavor

**Análise e visualização de dados**

Gabriela Custódio e Alexandre Cajazeira

In [1]:
# importar bibliotecas
import pandas as pd
import simplejson as json
import datetime
import plotly.express as px

## 1. Coletando dados <a name="Coleta"></a>

Neste primeiro passo, vamos raspar dados de internações em leitos de UTI e Enfermaria tanto para Covid-19 quanto para SRAG. Nesse caso, as informações estão completas e contemplam todo o período da pandemia. Elas estão disponíveis no IntegraSUS, plataforma da Secretaria da Saúde do Estado do Ceará (Sesa), no [Painel Histórico de Internações por SRAG / COVID-19](https://integrasus.saude.ce.gov.br/#/indicadores/indicadores-coronavirus/historico-internacoes-covid).

Ao executar esse passo, os dados finais estarão atualizados e serão diferentes do que foi coletado pela equipe do DATADOC.

In [2]:
#  criando a função consultaIntegra

def consultaIntegra(data):
    url = 'https://integrasus.saude.ce.gov.br/api/historico-internacoes-covid/taxa-ocupacao-unidade-perfil-leito-tipos?data={}&unidade=&municipio=&idRegiaoSaude=&idMacrorregiao=&idMacrorregiao=&tipo=TODOS&tipoUnidade='.format(data)
    df = pd.read_json(url)
    df['data'] = data
    return df;

In [3]:
# criando série com dias entre duas datas

pdRange = pd.date_range(start=pd.to_datetime('2020-01-01'), end=datetime.datetime.today()).to_series()
print(f'Número de dias: {pdRange.shape[0]}')

Número de dias: 846


In [4]:
# fazendo uma consulta para cada data e concatenando tudo em um único dataset;

dados_total = pd.concat([consultaIntegra(x.strftime("%Y-%m-%d")) for x,y in pdRange.iteritems()], ignore_index=True)

In [5]:
# salvando em .csv com o seguinte padrão: '2022 + xx(mes) + xx(dia) _ xxh (hora) + xxm (minuto) + _dados_internacao_total.csv'

dados_total.to_csv('20220425_21h12m_dados_internacao_total.csv', sep=',', encoding='utf-8', index=False)

## 2. Lendo dados coletados <a name="Leitura"></a>

In [20]:
# importando o arquivo salvo no passo anterior

total_leitos = pd.read_csv('20220425_21h12m_dados_internacao_total.csv')

In [21]:
# criando a coluna 'tipo_agravo'

total_leitos['tipo_agravo'] = 'COVID-19 + SRAG (NÃO COVID-19)'

## 3. Criando novas variáveis para análises de leitos infantis <a name="Variaveis"></a>

In [22]:
# leitos TOTAL (SRAG + COVID) desde o início da pandemia

# ORDENAR COLUNAS
leitos_infantis = total_leitos[['data', 'unidade', 'tipoLeito', 'tipo_agravo',
                                'leitosAtivosInfantil', 'leitosOcupadosInfantil']]


# RENOMEAR COLUNAS
leitos_infantis.rename(columns={'tipoLeito': 'tipo_leito',
                                'leitosAtivosInfantil': 'leitos_ativos_infantil',
                                'leitosOcupadosInfantil': 'leitos_ocupados_infantil'}, inplace=True)


# FILTRAR POR TIPO DE LEITO: ENFERMARIA OU UTI
leitos_infantis_enfermaria = leitos_infantis[leitos_infantis['tipo_leito'] == 'ENFERMARIA']
leitos_infantis_uti = leitos_infantis[leitos_infantis['tipo_leito'] == 'UTI']

C:\Users\55859\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
# AGRUPAR DADOS POR DATA, TIPO DE LEITO E TIPO DE AGRAVO
groupby_leitos_infantis_enfermaria = leitos_infantis_enfermaria.groupby(by=['data', 'tipo_leito', 'tipo_agravo']).sum().reset_index()
groupby_leitos_infantis_uti = leitos_infantis_uti.groupby(by=['data', 'tipo_leito', 'tipo_agravo']).sum().reset_index()


# CÁLCULO DA MÉDIA MÓVEL DE LEITOS ATIVOS
# em UTI
groupby_leitos_infantis_uti['mm_leitos_ativos_infantil'] = round(groupby_leitos_infantis_uti['leitos_ativos_infantil'].rolling(7).mean(),1)

# em Enfermaria
#leitos infantis
groupby_leitos_infantis_enfermaria['mm_leitos_ativos_infantil'] = round(groupby_leitos_infantis_enfermaria['leitos_ativos_infantil'].rolling(7).mean(),1)


# CÁLCULO DA MÉDIA MÓVEL DE LEITOS OCUPADOS (PACIENTES INTERNADOS) NA ENFERMARIA
groupby_leitos_infantis_enfermaria['mm_leitos_ocupados_infantil'] = round(groupby_leitos_infantis_enfermaria['leitos_ocupados_infantil'].rolling(7).mean(),1)


# CÁLCULO DA MÉDIA MÓVEL DE LEITOS OCUPADOS (PACIENTES INTERNADOS) NA UTI
groupby_leitos_infantis_uti['mm_leitos_ocupados_infantil'] = round(groupby_leitos_infantis_uti['leitos_ocupados_infantil'].rolling(7).mean(),1)


# CÁLCULO DA TAXA DE OCUPAÇÃO DOS LEITOS DE ENFERMARIA
# leitos infantil
groupby_leitos_infantis_enfermaria['ocup_leitos_infantil'] = round((groupby_leitos_infantis_enfermaria['leitos_ocupados_infantil']/groupby_leitos_infantis_enfermaria['leitos_ativos_infantil'])*100,2)


# CÁLCULO DA TAXA DE OCUPAÇÃO DOS LEITOS DE UTI
groupby_leitos_infantis_uti['ocup_leitos_infantil'] = round((groupby_leitos_infantis_uti['leitos_ocupados_infantil']/groupby_leitos_infantis_uti['leitos_ativos_infantil'])*100,2)


# REORDENAR COLUNAS
# ENFERMARIA
groupby_leitos_infantis_enfermaria = groupby_leitos_infantis_enfermaria[['data', 'tipo_leito', 'tipo_agravo',
                                                                         'leitos_ativos_infantil', 'mm_leitos_ativos_infantil',
                                                                         'leitos_ocupados_infantil', 'ocup_leitos_infantil',
                                                                         'mm_leitos_ocupados_infantil']]

# UTI
groupby_leitos_infantis_uti = groupby_leitos_infantis_uti[['data', 'tipo_leito', 'tipo_agravo',
                                                           'leitos_ativos_infantil', 'mm_leitos_ativos_infantil',
                                                           'leitos_ocupados_infantil', 'ocup_leitos_infantil',
                                                           'mm_leitos_ocupados_infantil']]

# CONCATENAR
concat_leitos_infantis = pd.concat([groupby_leitos_infantis_enfermaria, groupby_leitos_infantis_uti])

## 4. Tratamento <a name="Tratamento"></a>

In [24]:
# preenchendo células vazias com 0.0

concat_leitos_infantis.fillna(0.0, inplace=True)

In [25]:
# transformando coluna 'data' em formato datetime

concat_leitos_infantis['data'] = pd.to_datetime(concat_leitos_infantis['data'], format='%Y-%m-%d')

In [26]:
# reordenando as colunas

concat_leitos_infantis = concat_leitos_infantis[['data', 'tipo_leito', 'tipo_agravo',
                                                 'leitos_ativos_infantil', 'leitos_ocupados_infantil',
                                                 'ocup_leitos_infantil', 'mm_leitos_ocupados_infantil',
                                                 'mm_leitos_ativos_infantil']]

In [27]:
# ordenando dados pela data mais recente

concat_leitos_infantis.sort_values(by='data', ascending=False).head(6)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
725,2022-04-25,UTI,COVID-19 + SRAG (NÃO COVID-19),44.0,35.0,79.55,32.1,40.1
725,2022-04-25,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),288.0,274.0,95.14,213.9,233.1
724,2022-04-24,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),79.0,78.0,98.73,212.7,233.3
724,2022-04-24,UTI,COVID-19 + SRAG (NÃO COVID-19),26.0,26.0,100.00,32.3,40.0
723,2022-04-23,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),93.0,73.0,78.49,211.0,231.6
723,2022-04-23,UTI,COVID-19 + SRAG (NÃO COVID-19),33.0,25.0,75.76,32.4,40.3


## 5. Análises <a name="Analise"></a>

In [28]:
# calcular variação entre a mm_leitos_ocupados_infantil no dia 23/12/2021,
# quando a Ômicron foi confirmada no Ceará, e na data mais recente

media_movel_atual = datetime.datetime.today()

VI = concat_leitos_infantis.loc[(concat_leitos_infantis['data'] == '2021-12-23')].reset_index()
VF = concat_leitos_infantis.loc[(concat_leitos_infantis['data'] == '2022-04-25')].reset_index() # altere DATA em VF

print('ordem em que os valores aparecem')
print(concat_leitos_infantis.loc[(concat_leitos_infantis['data'] == '2022-04-25')]['tipo_leito'])

coluna = 'mm_leitos_ocupados_infantil'  #selecione a coluna para o cálculo da variação

print('média móvel em 23/12/2021')
print(VI[coluna])

print('média móvel atual, em:', media_movel_atual)
print(VF[coluna])

print('variação da média móvel')
var = round((VF[coluna]/VI[coluna]-1)*100,2)
var

ordem em que os valores aparecem
725    ENFERMARIA
725           UTI
Name: tipo_leito, dtype: object
média móvel em 23/12/2021
0    75.7
1     9.7
Name: mm_leitos_ocupados_infantil, dtype: float64
média móvel atual, em: 2022-04-26 15:15:18.701377
0    213.9
1     32.1
Name: mm_leitos_ocupados_infantil, dtype: float64
variação da média móvel


0    182.56
1    230.93
Name: mm_leitos_ocupados_infantil, dtype: float64

**Nota:** Na visualização a seguir, é possível perceber que a maior média móvel de ocupação nos leitos infantis de enfermaria, desde o início da pandemia, foi registrada na data mais recente (25/04/2022)

In [29]:
# gerando visualização com a série histórica da média móvel de leitos infantis ocupados

px.line(concat_leitos_infantis, x='data', y='mm_leitos_ocupados_infantil', color='tipo_leito').show()

In [30]:
# confirmando o dia com maior média móvel de ocupação em leitos infantis de enfermaria

concat_leitos_infantis[concat_leitos_infantis['tipo_leito'] == 'ENFERMARIA'].sort_values(by='mm_leitos_ocupados_infantil', ascending=False).head(1)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
725,2022-04-25,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),288.0,274.0,95.14,213.9,233.1


In [31]:
# agora, encontrando o dia com maior média móvel de ocupação em leitos infantis na UTI

concat_leitos_infantis[concat_leitos_infantis['tipo_leito'] == 'UTI'].sort_values(by='mm_leitos_ocupados_infantil', ascending=False).head(1)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
638,2022-01-28,UTI,COVID-19 + SRAG (NÃO COVID-19),54.0,38.0,70.37,47.4,60.7


## 5.1 Período após a identificação da Ômicron no Estado do Ceará <a name="Omicron"></a>

In [32]:
# filtrando dataframe pelo período de tempo desde que a variante Ômicron foi identificada no Ceará

df_filtro_omicron = concat_leitos_infantis[concat_leitos_infantis['data'] >= '2021-12-23']
df_filtro_omicron.head()

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
602,2021-12-23,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),170.0,73.0,42.94,75.7,148.4
603,2021-12-24,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),47.0,29.0,61.70,66.0,131.7
604,2021-12-25,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),39.0,21.0,53.85,64.7,131.6
605,2021-12-26,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),40.0,25.0,62.50,55.6,114.1
606,2021-12-27,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),157.0,73.0,46.50,52.7,113.4


**Nota:** A seguir, podemos ver que, desde que a variante Ômicron do Sars-Cov-2 foi identificada no Ceará, no dia 23 de dezembro de 2021, os 10 dias com maiores médias móveis do número de leitos de enfermaria voltados para o público infantil que estão ocupados (ou seja, os 10 dias com maiores médias móveis da quantidade de crianças internadas) foram registrados em abril de 2022. O maior registro foi o desta segunda-feira, 25/04/2022.

In [33]:
# encontrando os dias com maiores médias móveis de internação de crianças em leitos de enfermaria desde
# que a variante Ômicron foi identificada no Ceará, em 23/12/2021

df_filtro_omicron[df_filtro_omicron['tipo_leito'] == 'ENFERMARIA'].sort_values(by='mm_leitos_ocupados_infantil', ascending=False).head(10)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
725,2022-04-25,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),288.0,274.0,95.14,213.9,233.1
724,2022-04-24,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),79.0,78.0,98.73,212.7,233.3
723,2022-04-23,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),93.0,73.0,78.49,211.0,231.6
722,2022-04-22,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),312.0,275.0,88.14,209.4,228.7
721,2022-04-21,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),311.0,277.0,89.07,199.9,215.7
707,2022-04-07,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),233.0,207.0,88.84,194.9,213.1
708,2022-04-08,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),215.0,200.0,93.02,193.4,211.9
720,2022-04-20,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),282.0,266.0,94.33,191.9,204.9
706,2022-04-06,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),224.0,208.0,92.86,191.4,212.7
709,2022-04-09,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),68.0,59.0,86.76,190.6,207.9


**Nota:**
Nos números absolutos de crianças internadas em enfermarias para Covid/SRAG, o dia 25/04 foi o 3º com mais casos. O primeiro foi o dia 21/04/22 e o segundo, 22/04/22

In [34]:
# encontrando os dias com maiores números absolutos de leitos infantis ocupados nas enfermarias desde
# que a variante Ômicron foi identificada no Ceará, em 23/12/2021

df_filtro_omicron[df_filtro_omicron['tipo_leito'] == 'ENFERMARIA'].sort_values(by='leitos_ocupados_infantil', ascending=False).head(10)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
721,2022-04-21,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),311.0,277.0,89.07,199.9,215.7
722,2022-04-22,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),312.0,275.0,88.14,209.4,228.7
725,2022-04-25,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),288.0,274.0,95.14,213.9,233.1
720,2022-04-20,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),282.0,266.0,94.33,191.9,204.9
718,2022-04-18,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),289.0,266.0,92.04,160.9,174.0
719,2022-04-19,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),267.0,254.0,95.13,188.1,201.1
713,2022-04-13,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),256.0,240.0,93.75,168.7,183.0
704,2022-04-04,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),258.0,235.0,91.09,185.4,217.3
703,2022-04-03,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),238.0,230.0,96.64,161.7,194.3
714,2022-04-14,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),235.0,221.0,94.04,170.7,183.3


**Nota:** Nos leitos de UTI, os dias com maiores índices de internação de crianças ocorreu entre janeiro e fevereiro de 2022

In [35]:
# encontrando os dias com maiores médias móveis de internação de crianças em UTI desde
# que a variante Ômicron foi identificada no Ceará, em 23/12/2021

df_filtro_omicron[df_filtro_omicron['tipo_leito'] == 'UTI'].sort_values(by='mm_leitos_ocupados_infantil', ascending=False).head(10)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
638,2022-01-28,UTI,COVID-19 + SRAG (NÃO COVID-19),54.0,38.0,70.37,47.4,60.7
637,2022-01-27,UTI,COVID-19 + SRAG (NÃO COVID-19),71.0,50.0,70.42,47.3,59.7
639,2022-01-29,UTI,COVID-19 + SRAG (NÃO COVID-19),44.0,37.0,84.09,46.4,59.0
636,2022-01-26,UTI,COVID-19 + SRAG (NÃO COVID-19),65.0,51.0,78.46,46.1,56.4
640,2022-01-30,UTI,COVID-19 + SRAG (NÃO COVID-19),45.0,32.0,71.11,45.1,59.1
635,2022-01-25,UTI,COVID-19 + SRAG (NÃO COVID-19),69.0,56.0,81.16,44.6,54.6
654,2022-02-13,UTI,COVID-19 + SRAG (NÃO COVID-19),65.0,48.0,73.85,43.7,70.4
641,2022-01-31,UTI,COVID-19 + SRAG (NÃO COVID-19),63.0,41.0,65.08,43.6,58.7
655,2022-02-14,UTI,COVID-19 + SRAG (NÃO COVID-19),72.0,43.0,59.72,43.0,69.6
649,2022-02-08,UTI,COVID-19 + SRAG (NÃO COVID-19),78.0,47.0,60.26,42.4,68.7


**Nota:** Foi em 25/01/22 o registro de maior quantidade de crianças internadas em UTI pediátrica para Covid/SRAG

In [36]:
# encontrando os dias com maiores números absolutos de leitos infantis ocupados nas UTIs desde
# que a variante Ômicron foi identificada no Ceará, em 23/12/2021

df_filtro_omicron[df_filtro_omicron['tipo_leito'] == 'UTI'].sort_values(by='leitos_ocupados_infantil', ascending=False).head(10)

,data,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil,ocup_leitos_infantil,mm_leitos_ocupados_infantil,mm_leitos_ativos_infantil
635,2022-01-25,UTI,COVID-19 + SRAG (NÃO COVID-19),69.0,56.0,81.16,44.6,54.6
634,2022-01-24,UTI,COVID-19 + SRAG (NÃO COVID-19),66.0,52.0,78.79,41.6,50.6
636,2022-01-26,UTI,COVID-19 + SRAG (NÃO COVID-19),65.0,51.0,78.46,46.1,56.4
637,2022-01-27,UTI,COVID-19 + SRAG (NÃO COVID-19),71.0,50.0,70.42,47.3,59.7
643,2022-02-02,UTI,COVID-19 + SRAG (NÃO COVID-19),82.0,49.0,59.76,41.9,62.9
648,2022-02-07,UTI,COVID-19 + SRAG (NÃO COVID-19),78.0,48.0,61.54,42.3,69.1
654,2022-02-13,UTI,COVID-19 + SRAG (NÃO COVID-19),65.0,48.0,73.85,43.7,70.4
649,2022-02-08,UTI,COVID-19 + SRAG (NÃO COVID-19),78.0,47.0,60.26,42.4,68.7
650,2022-02-09,UTI,COVID-19 + SRAG (NÃO COVID-19),76.0,46.0,60.53,42.0,67.9
642,2022-02-01,UTI,COVID-19 + SRAG (NÃO COVID-19),81.0,46.0,56.79,42.1,60.4


## 5.2 Identificando hospitais com as internações <a name="Hospitais"></a>

In [37]:
# encontrando os hospitais que, neste dia 25/04/2022, tinham leitos infantis específicos para SRAG/COVID
# em ordem decrescente do número de crianças internadas em cada um deles

leitos_infantis_hospitais_2504 = leitos_infantis[leitos_infantis['data'] == '2022-04-25']
leitos_infantis_hospitais_2504[leitos_infantis_hospitais_2504['leitos_ativos_infantil'] != 0.0].sort_values(by='leitos_ocupados_infantil', ascending=False)

,data,unidade,tipo_leito,tipo_agravo,leitos_ativos_infantil,leitos_ocupados_infantil
96738,2022-04-25,2526638 - SOPAI HOSPITAL INFANTIL,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),196.0,196.0
96754,2022-04-25,2563681 - HIAS HOSPITAL INFANTIL ALBERT SABIN,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),58.0,58.0
96755,2022-04-25,2563681 - HIAS HOSPITAL INFANTIL ALBERT SABIN,UTI,COVID-19 + SRAG (NÃO COVID-19),25.0,25.0
96769,2022-04-25,2785900 - HGWA - HOSPITAL GERAL DR WALDEMAR AL...,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),18.0,18.0
96739,2022-04-25,2526638 - SOPAI HOSPITAL INFANTIL,UTI,COVID-19 + SRAG (NÃO COVID-19),10.0,10.0
96753,2022-04-25,2562871 - HOSPITAL SAO RAIMUNDO,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),3.0,1.0
96774,2022-04-25,3242587 - HOSPITAL REGIONAL UNIMED,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),3.0,1.0
96752,2022-04-25,2562871 - HOSPITAL SAO RAIMUNDO,UTI,COVID-19 + SRAG (NÃO COVID-19),3.0,0.0
96770,2022-04-25,3001113 - HOSPITAL OTOCLINICA,ENFERMARIA,COVID-19 + SRAG (NÃO COVID-19),10.0,0.0
96771,2022-04-25,3001113 - HOSPITAL OTOCLINICA,UTI,COVID-19 + SRAG (NÃO COVID-19),6.0,0.0


## 6. Salvar arquivos em .csv <a name="Arquivo"></a>

In [38]:
# Renomeando colunas para preparar tabela para gráfico no Flourish

concat_leitos_infantis.rename(columns={'tipo_leito': 'Tipo de leito',
                                       'tipo_agravo': 'Tipo de agravo',
                                       'leitos_ativos_infantil': 'Número de leitos infantil ativos',
                                       'leitos_ocupados_infantil': 'Número de leitos infantil ocupados',
                                       'ocup_leitos_infantil': 'Taxa de ocupação (%) de leitos infantil',
                                       'mm_leitos_ocupados_infantil': 'Média móvel do número de crianças internadas',
                                       'mm_leitos_ativos_infantil': 'Média móvel do número de leitos para crianças ativos'}, inplace=True)

In [39]:
# salvando concat_leitos_infantis em .csv

concat_leitos_infantis.to_csv('df_arquivo_leitos_infantis_grafico.csv', sep=',', encoding='utf-8', index=False)

In [40]:
# Salvando o dataframe feito a partir do filtro de tempo desde a identificação da variante Ômicron

df_filtro_omicron.to_csv('df_filtro_omicron.csv', sep=',', encoding='utf-8', index=False)

In [41]:
# Salvando os dataframes com os leitos ativos e ocupados por unidade de saúde

leitos_infantis_hospitais_2504.to_csv('df_leitos_por_hospital.csv', sep=',', encoding='utf-8', index=False)